In [1]:
import numpy as np
from astropy.convolution import Gaussian2DKernel
from astropy.io import fits
import matplotlib.pyplot as plt
import scipy.signal as sig 

# These are how I like to format my matplotlib plots. Feel Free to remove/comment out
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10.0, 10.0)     # set figure size
mpl.rcParams['font.family'] = 'Serif'      # set font
mpl.rcParams['axes.unicode_minus'] = False
mpl.rcParams['font.size'] = 17
mpl.rcParams['legend.fontsize'] = 15
mpl.rcParams['xtick.top'] = True  
mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['xtick.major.size'] = 18
mpl.rcParams['xtick.major.pad'] = 14
mpl.rcParams['ytick.major.pad'] = 14

mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['xtick.minor.size'] = 10
mpl.rcParams['ytick.major.size'] = 18
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['ytick.minor.size'] = 10
mpl.rcParams['xtick.labelsize'] = 25
mpl.rcParams['ytick.labelsize'] = 25

# Simulated test cube

### The cube will have 1000 layers (acting as the wavelength axis)

### There will be some ~ semi complex resolved emission background that is only present for certain wavelength values. This complex emission will be a pseudo extended emission line region. The emission line will be in the shape of a 1D Gaussian

### We will throw in point sources at random places overlapping with the resolved emission, in similar fashion to bright QSOs or stars. For simplicity here, they will just have some constant flux value, rather than a true realistic spectrum

### We will convolve the cube with a 2D Gaussian and throw in a noise model to each wavelength layer.

### Control the resolution of the grid to understand how the specific parameters of cubecarve might have to change to get good performance

In [5]:
# simple gaussian 1D function

def gaussian(x, mu, sig):
    """
    Calculates the 1D Gaussian probability density function (PDF).
    
    Args:
        x (numpy.array): The x-values at which to evaluate the PDF.
        mu (float): The mean (center) of the distribution.
        sig (float): The standard deviation (width) of the distribution.
        
    Returns:
        numpy.array: The corresponding y-values of the Gaussian curve.
    """
    # The formula for the normal distribution PDF
    return (1.0 / (np.sqrt(2.0 * np.pi) * sig) * 
            np.exp(-np.power((x - mu) / sig, 2.0) / 2))


In [24]:

def make_cube(filename,filename_model, Nx,Ny): 

    # Image grid

    nz, ny, nx = 1000, Ny, Nx
    # Make 1d gaussian emission line mask. mean = 200, width = 5
    line_mask = gaussian(np.arange(0,nz),200,5)
    cube = np.zeros((nz,nx,ny))
    cube_resolved = np.zeros((nz,Ny, Nx))
    MEAN = 0.0           # The mean (center) of the noise distribution
    STD_DEV = 0.02      # The standard deviation (spread) of the noise
    ARRAY_SHAPE = (Ny, Nx) # The desired dimensions (height, width) of the 2D array

    # Generate the 2D Gaussian noise model ---
    gaussian_noise_2d = np.random.normal(MEAN, STD_DEV, ARRAY_SHAPE)

    for i in range(nz):
        y, x = np.indices((ny, nx))

        # Center of the structure
        x0, y0 = nx // 2, ny // 2

        # Elliptical scaling (controls ellipticity)
        a = 2      # x-axis scaling
        b = 1.0      # y-axis scaling

        # Shifted coordinates
        xp = (x - x0) / a
        yp = (y - y0) / b

        # Elliptical radius and angle
        r = np.sqrt(xp**2 + yp**2)
        theta = np.arctan2(yp, xp)

        # -----------------------------
        # Low-frequency edge modulation
        # -----------------------------
        # Controls the waviness of the ellipse boundary
        wave_amp = 0.2        # strength of the edge perturbation
        wave_mode = 15         # number of lobes (low-frequency)
        r_edge = 50.0          # characteristic radius of the ellipse

        r_mod = r_edge * (1 + wave_amp * np.cos(wave_mode * theta))

        # -----------------------------
        # Radial brightness profile
        # -----------------------------
        # Bright inside, fading outward
        sigma = 18.0
        radial_profile = np.exp(-0.5 * (r / sigma)**2)

        # Soft edge using a logistic transition
        edge_width = 1.0
        edge_mask = 1.0 / (1.0 + np.exp((r - r_mod) / edge_width))

        kernel = Gaussian2DKernel(2.5).array


        # Final image
        image = radial_profile * edge_mask * line_mask[i] * 3

        ##### Cone Emission Add On

        y, x = np.indices((ny, nx))

        # Center of the image
        x0, y0 = nx // 2, ny // 2
        dx = x - x0
        dy = y - y0

        # -----------------------------
        # Position angle (degrees)
        # -----------------------------
        PA_deg = 104.0            # change this to rotate the cones
        PA = np.deg2rad(PA_deg)

        # Rotate coordinates
        x_rot = dx * np.cos(PA) + dy * np.sin(PA)
        y_rot = -dx * np.sin(PA) + dy * np.cos(PA)

        # -----------------------------
        # Bi-conical geometry
        # -----------------------------
        opening_angle = np.deg2rad(50.0)   # half-opening angle of each cone

        r = np.sqrt(x_rot**2 + y_rot**2)

        # Angle away from cone axis (absolute gives two opposite cones)
        theta = np.arctan2(np.abs(y_rot), np.abs(x_rot))

        # Cone mask
        cone_mask = theta < opening_angle

        # -----------------------------
        # Radial brightness profile
        # -----------------------------
        # Suppress very center + fade outward
        r_inner = 1.0
        r_scale = 30.0

        radial_profile = np.exp(-r / r_scale) * (1.0 - np.exp(-r / r_inner))

        # -----------------------------
        # Soft angular edges
        # -----------------------------
        angular_soft = np.exp(-0.5 * (theta / opening_angle)**4)

        # -----------------------------
        # Final image
        # -----------------------------
        image += cone_mask * radial_profile * angular_soft * line_mask[i] * 2


        cube_resolved[i] = sig.fftconvolve(image,kernel,mode='same') + gaussian_noise_2d
        image[55,41] += 10
        image[45,55] += 15
        image = sig.fftconvolve(image,kernel,mode='same')
        image += gaussian_noise_2d

        cube[i] = image

    hdu = fits.PrimaryHDU(data=cube)
    hdu.writeto(filename,overwrite=True)
    print('fits file '+filename+' saved')

    hdu = fits.PrimaryHDU(data=cube_resolved)
    hdu.writeto(filename_model,overwrite=True)
    print('fits file '+filename+' saved')

    

In [20]:
filename = '../data/testcube_lowres.fits'
filename_no_stars = '../data/testcube_nostars_lowres.fits'
ny, nx = 30, 30
make_cube(filename,filename_no_stars,ny,nx)

fits file ../data/testcube_lowres.fits saved
fits file ../data/testcube_lowres.fits saved


In [25]:
# higher res, 100x100 grid

filename = '../data/testcube_highres.fits'
filename_no_stars = '../data/testcube_nostars_highres.fits'
ny, nx = 100, 100
make_cube(filename,filename_no_stars,ny,nx)

fits file ../data/testcube_highres.fits saved
fits file ../data/testcube_highres.fits saved
